# Task 1 - Preguntas Teóricas
1. ¿Por qué el modelo de Naive Bayes se le considera “naive”?

Se consideran Naive debido a que en ellos se asume que las variables predictoras son independientes entre sí. Es decir, que la presencia de una cierta característica en un conjunto de datos no está en absoluto relacionada con la presencia de cualquier otra característica.

Victor Roman. (2019). Algoritmos Naive Bayes: Fundamentos e Implementación. Medium. https://medium.com/datos-y-ciencia/algoritmos-naive-bayes-fudamentos-e-implementaci%C3%B3n-4bcb24b307f


2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)
3. 

4. Investigue sobre Random Forest y responda
    a. ¿Qué tipo de ensemble learning es este modelo?
      Pertenece al grupo de Bagging (Bootstrap Aggregating) ya que divide la serie de datos en varios subconjuntos compuestos aleatoriamente de muestras, y se entrena un modelo en cada subconjunto, esto significa que habrá tantos modelos como subconjuntos creados. Luego se combinan todos los resultados de los modelos para obtener un resultado final.
    b. ¿Cuál es la idea general detrás de Random Forest?
        Construye un conjunto diverso de árboles de decisión, donde cada uno de estos árboles se entrena en una muestra aleatoria del conjunto de datos. Un tercio se reserva como datos de prueba y a esto se le conoce como muestra fuera de la empresa (oob). Luego, se inyecta otra instancia de aleatoriedad a través del agrupamiento de características, lo que agrega más diversidad al conjunto de datos y reduce la correlación entre los árboles de decisión. La determinación de la predicción depende del tipo del problema, para una tarea de regresión se promedia los árboles de decisión individuales, mientras que para un problema de clasificación se utiliza un voto mayoritario, es decir, la variable categórica más frecuente, arrojará la clase predicha.
    c. ¿Por qué se busca baja correlación entre los árboles de Random Forest?
        La baja correlación reduce el sobreajuste y favorece la precisión del modelo, ya que asegura que cada árbol creado aporte información única. Ya que como el nombre del modelo lo indica (naive), se asume que las variables son independientes entre sí.

¿Qué es un bosque aleatorio? (s/f). Ibm.com. Recuperado el 25 de enero de 2024, de https://www.ibm.com/mx-es/topics/random-forest

Random Forest: Bosque aleatorio. Definición y funcionamiento. (2022, enero 25). Formación en ciencia de datos | Datascientest.com. https://datascientest.com/es/random-forest-bosque-aleatorio-definicion-y-funcionamiento

# Task 2 - Naive Bayes: Clasificador de Mensajes Ham/Spam
## Task 2.1 - Lectura y limpieza del dataset

In [98]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [99]:
file = open('entrenamiento.txt', 'r')
dataset = []
for line in file:
    print(line.split('    '))
    dataset.append(line.split('    '))

['ham', '"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,\n']
['ham', 'Ok lar... Joking wif u oni...,,,\n']
['spam', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,\n"]
['ham', 'U dun say so early hor... U c already then say...,,,\n']
['ham', '"Nah I don\'t think he goes to usf, he lives around here though",,,\n']
['spam', '"FreeMsg Hey there darling it\'s been 3 week\'s now and no word back! I\'d like some fun you up for it still? Tb ok! XxX std chgs to send, ï¿½1.50 to rcv",,,\n']
['ham', 'Even my brother is not like to speak with me. They treat me like aids patent.,,,\n']
['ham', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,\n"]
['spam', 'WINNER!! As a valued network customer you have been

In [100]:
dataset = pd.DataFrame(dataset)
dataset

,0,1
0,ham,"""Go until jurong point, crazy.. Available only..."
1,ham,"Ok lar... Joking wif u oni...,,,\n"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"""Nah I don't think he goes to usf, he lives ar..."
5,spam,"""FreeMsg Hey there darling it's been 3 week's ..."
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### Limpiar el dataset
- Limpiar los caracteres especiales
- Misma nomenclatura (todo mayúsculas o minúsculas)

In [101]:
# Limpiar caracteres especiales

for line in dataset.values:
    line[1] = line[1].lower()
    for c in line[1]:
        if c.isalpha() == False and c != ' ':
            line[1] = line[1].replace(c, '')
        
    line[1] = ' '.join([p for p in line[1].split(' ') if p != '']) # Quitar espacios en blanco

dataset

,0,1
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final ...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...
5,spam,freemsg hey there darling its been weeks now a...
6,ham,even my brother is not like to speak with me t...
7,ham,as per your request melle melle oru minnaminun...
8,spam,winner as a valued network customer you have b...
9,spam,had your mobile months or more u r entitled to...


In [102]:
# Codificamos la variable categoricas en 1 = ham y 0 = spam  
def codif_y_ligar(dataframe_original, variables_por_codificar):
    """Codifica las variables categóricas y las une al dataframe original"""
    dummies = pd.get_dummies(dataframe_original[[variables_por_codificar]])
    resultado = pd.concat([dataframe_original, dummies], axis = 1)
    resultado = resultado.drop([variables_por_codificar], axis = 1)
    return resultado

variables_a_codificar = [0] # Variables categóricas. Status tiene valores 'spam' y 'ham'
for variable in variables_a_codificar:
    dataset = codif_y_ligar(dataset, variable)

In [108]:
X = dataset.iloc[:, 0].values
y = dataset.iloc[:, 1].values

In [113]:
y # HAM ES TRUE

array([ True,  True, False,  True,  True, False,  True,  True, False,
       False,  True, False, False,  True,  True])

In [110]:
# separar data en datos de entrenamiento y datos de prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [111]:
X_train

array(['had your mobile months or more u r entitled to update to the latest colour mobiles with camera for free call the mobile update co free on',
       'i have a date on sunday with will',
       'nah i dont think he goes to usf he lives around here though',
       'free entry in a wkly comp to win fa cup final tkts st may text fa to to receive entry questionstd txt ratetcs apply overs',
       'ive been searching for the right words to thank you for this breather i promise i wont take your help for granted and will fulfil my promise you have been wonderful and a blessing at all times',
       'im gonna be home soon and i dont want to talk about this stuff anymore tonight k ive cried enough today',
       'as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press to copy your friends callertune',
       'six chances to win cash from to pounds txt csh and send to cost pday days tsandcs apply reply hl info',
       'u dun 

## Task 2.2 - Construcción del modelo

In [114]:
# Crear el modelo de Naive Bayes
K = 1